In [24]:
import collections
import datetime
import json
import math
import os
import random
import copy
import time
import tempfile
import subprocess
import torch
import torch.utils.data
import torchvision.transforms
import numpy as np
import visdom
import scipy
import einops
import json
from geomloss import SamplesLoss
from torch import nn, optim
from collections import defaultdict
from torch.optim.optimizer import Optimizer, required
from math import sqrt
from functools import partial, lru_cache
from torch.nn import functional as F
from torch.nn import Parameter
from PIL import Image
import matplotlib.pyplot as plt

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [25]:
#dataset defined by my self
class WarriorDataset(torch.utils.data.Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_names = os.listdir(folder_path)
        if transform:
            self.transform = torchvision.transforms.Compose([
                torchvision.transforms.Resize((70, 70)),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0, 0, 0), (1, 1, 1))
            ])
    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, index):
        image_name = self.image_names[index]
        image_path = os.path.join(self.folder_path, image_name)
        image = Image.open(image_path)
        image = image.convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image
    
def cycle(iterable):
    while True:
        for x in iterable:
            yield x

In [17]:
folder_path = os.getcwd() + "/Pictures/Warrior"
dataset = WarriorDataset(folder_path, transform=True)
train_loader = torch.utils.data.DataLoader(
    dataset,
shuffle=True, batch_size=16, drop_last=True)
train_iterator = iter(cycle(train_loader))

print(dataset[50])

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


In [18]:
args = {
    'width': 32,
    'dataset': 'easy_worrior',
    'n_channels': 3,
    'n_classes': 10,
    'batch_size': 16,
    'vid_batch': 16,
    'latent_dim': 8,  # lower is better modelling but worst interpolation freedom
    'lr': 0.005,
    'log_every': 2
}

In [19]:
if args['dataset'] == 'easy_worrior': # Test case
    xb = next(train_iterator)
    xb = xb.to(device)
else:
    xb,cb = next(train_iterator)
    xb,cb = xb.to(device), cb.to(device)



In [20]:
class Decoder(nn.Module):
    def __init__(self, latent_dim, n_channels):
        super(Decoder, self).__init__()
        self.f1 = nn.Sequential(
            nn.LazyConvTranspose2d(512, 4, stride=1, padding=0),
            nn.LazyBatchNorm2d(),
            nn.ReLU())  # 4x4
        self.f2 = nn.Sequential(
            nn.LazyConvTranspose2d(256, 4, stride=2, padding=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU())  # 8x8
        self.f3 = nn.Sequential(
            nn.LazyConvTranspose2d(128, 4, stride=2, padding=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU())  # 16x16
        self.f4 = nn.Sequential(
            nn.LazyConvTranspose2d(n_channels, 4, stride=2, padding=1),
            nn.Sigmoid())  # 32x32

    def forward(self, z):
        x = self.f1(z)
        x = self.f2(x)
        x = self.f3(x)
        x = self.f4(x)
        # x = torch.nn.functional.softmax(x, dim=1)
        return x


net = Decoder(args['latent_dim'], args['n_channels']).to(device)
opt = torch.optim.Adam(net.parameters(), lr=args['lr'])
ot_loss_fn = SamplesLoss("sinkhorn", p=2, blur=0.001)

C:\Users\Nostalgia\anaconda3\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [21]:
def ot_loss(x, y):
    return ot_loss_fn(x.view(x.size(0), -1), y.view(y.size(0), -1))

In [22]:
logs = {}
logs['loss1'] = logs['loss2'] = logs['loss3'] = 0
logs['num_stats'] = 0

opt.zero_grad()

# p(x | z)
# p(x | z, p)

p_z = torch.randn(args['batch_size'], args['latent_dim'], 1, 1).to(device)
g = net(p_z)

loss = ot_loss(g, xb)  # ((g-xb)**2).mean()
loss.backward()
opt.step()

ValueError: Input samples 'x' and 'y' should have the same last dimension.

In [34]:
xb.size(3)

70